In [ ]:
!pip install ensemble_boxes

In [ ]:
import pandas as pd
pred_14cls = pd.read_csv('/content/final.csv')
pred_2cls = pd.read_csv('/content/2-cls test pred.csv')
pred = pd.merge(pred_14cls, pred_2cls, on = 'image_id', how = 'left')
thr = 0.98
def filter_2cls(row, thr=thr):
    if row['0']>thr:
        row['PredictionString'] = '14 1 0 0 1 1'
    return row
sub = pred.apply(filter_2cls, axis=1)
sub[['image_id', 'PredictionString']].to_csv('yolov5_fold0.csv',index = False)

In [ ]:
import numpy as np
import pandas as pd
from ensemble_boxes import nms, soft_nms, non_maximum_weighted, weighted_boxes_fusion
iou_thr = 0.5
skip_box_thr = 0.01
sigma = 0.1
#weight=[40, 40, 40, 40, 40, 40]
yolov5_fold0= pd.read_csv('/content/yolo_fold_0_TTA_submission_thr_0.86.csv')
yolov5_fold1= pd.read_csv('/content/yolov5_fold_1_TTA.csv')
yolov5_fold2= pd.read_csv('/content/yolov5_fold_2_TTA.csv')
yolov5_fold3= pd.read_csv('/content/yolov5_fold_3_TTA.csv')
yolov5_fold4= pd.read_csv('/content/yolov5_fold_4.csv')
detectron2_data =pd.read_csv('/content/submit_detectron2 (3).csv')
test_df =pd.read_csv('/content/test.csv')
image_id_lst = yolov5_fold0['image_id'].unique()
merged_df = pd.DataFrame(columns=['image_id', 'PredictionString'])
ensemble_filename = 'final_ensemnle_yolov5_detetron_efficient_model.csv'
def h_w_image(image_id): 
  height = test_df[test_df['image_id'] == image_id]['height'].values[0]
  width = test_df[test_df['image_id']  == image_id]['width'].values[0]
  return height,width

def convert_data(data,width,height):
  list_lb = []
  list_score=[]
  list_box =[]
  list_data =data.split(' ')
  for i in range(0,len(list_data),6):
    if (float(list_data[i])==0 or float(list_data[i])==3): # thêm loại bỏ class 3 và 0 dưới ngưỡng 0.3
      if float(list_data[i+1])>=0.05:
        list_lb.append(list_data[i])
        list_score.append(float(list_data[i+1]))
        x_min = float(list_data[i+2])/width
        y_min = float(list_data[i+3])/height
        x_max = float(list_data[i+4])/width
        y_max = float(list_data[i+5])/height
        list_box.append([x_min,y_min,x_max,y_max])
      else:
        continue
      continue
    else:
       list_lb.append(list_data[i])
       list_score.append(float(list_data[i+1]))
       x_min = float(list_data[i+2])/width
       y_min = float(list_data[i+3])/height
       x_max = float(list_data[i+4])/width
       y_max = float(list_data[i+5])/height
       list_box.append([x_min,y_min,x_max,y_max])
  return list_lb ,list_score,list_box
def convert_data_to_row(boxes, scores, labels):
    data_lst = []
    for i in range(len(boxes)):
        data_lst.append(str(int(labels[i])))
        data_lst.append(str(scores[i]))
        data_lst.append(str(boxes[i][0]))
        data_lst.append(str(boxes[i][1]))
        data_lst.append(str(boxes[i][2]))
        data_lst.append(str(boxes[i][3]))
    data = ' '.join(data_lst)
    return data
def scale_data(boxes,img_height,img_width):
    boxes[:, 0] = boxes[:, 0] * img_width
    boxes[:, 1] = boxes[:, 1] * img_height
    boxes[:, 2] = boxes[:, 2] * img_width
    boxes[:, 3] = boxes[:, 3] * img_height
    return boxes
def wbf(image_id):
  img_height ,img_width = h_w_image(image_id)
  list_lb ,list_score,list_box =[],[],[]

  yolov5_fold_0 = yolov5_fold0[yolov5_fold0['image_id'] == image_id]['PredictionString'].values[0]
  list_lb_pr ,list_score_pr,list_box_pr = convert_data(yolov5_fold_0,img_width,img_height)
  list_lb.append(list_lb_pr)
  list_score.append(list_score_pr)
  list_box.append(list_box_pr)

  yolov5_fold_1 = yolov5_fold1[yolov5_fold1['image_id'] == image_id]['PredictionString'].values[0]
  list_lb_pr ,list_score_pr,list_box_pr = convert_data(yolov5_fold_1,img_width,img_height)
  list_lb.append(list_lb_pr)
  list_score.append(list_score_pr)
  list_box.append(list_box_pr)

  yolov5_fold_2 = yolov5_fold2[yolov5_fold2['image_id'] == image_id]['PredictionString'].values[0]
  list_lb_pr ,list_score_pr,list_box_pr = convert_data(yolov5_fold_2,img_width,img_height)
  list_lb.append(list_lb_pr)
  list_score.append(list_score_pr)
  list_box.append(list_box_pr)
  
  yolov5_fold_3 = yolov5_fold3[yolov5_fold3['image_id'] == image_id]['PredictionString'].values[0]
  list_lb_pr ,list_score_pr,list_box_pr = convert_data(yolov5_fold_3,img_width,img_height)
  list_lb.append(list_lb_pr)
  list_score.append(list_score_pr)
  list_box.append(list_box_pr)

  yolov5_fold_4 = yolov5_fold4[yolov5_fold4['image_id'] == image_id]['PredictionString'].values[0]
  list_lb_pr ,list_score_pr,list_box_pr = convert_data(yolov5_fold_4,img_width,img_height)
  list_lb.append(list_lb_pr)
  list_score.append(list_score_pr)
  list_box.append(list_box_pr)

  detectron2 = detectron2_data[detectron2_data['image_id'] == image_id]['PredictionString'].values[0]
  list_lb_pr ,list_score_pr,list_box_pr = convert_data(detectron2,img_width,img_height)
  list_lb.append(list_lb_pr)
  list_score.append(list_score_pr)
  list_box.append(list_box_pr)

  boxes, scores, labels = weighted_boxes_fusion(list_box,list_score ,list_lb, weights=[15,5,5,5,15,10], iou_thr=iou_thr, skip_box_thr=skip_box_thr)
  boxes = scale_data(boxes, img_height, img_width)
  merged_data = convert_data_to_row(boxes, scores, labels)
  merged_data = pd.DataFrame([[image_id, merged_data]], columns=['image_id', 'PredictionString'])
  return merged_data
for image_id in image_id_lst:
    merged_data = wbf(image_id)
    merged_df = merged_df.append(merged_data, ignore_index=True)
merged_df.to_csv(ensemble_filename, index=False)

